In [1]:
from dotenv import load_dotenv

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

load_dotenv()

True

## Model I/O Essentials:

1. Language Models
2. Output parsers
3. Prompts

### Language Models

In [19]:
from langchain_ollama import ChatOllama
from langchain_community.llms import HuggingFaceHub

llama_llm = ChatOllama(model='llama3.2:1b', base_url='http://localhost:11434')

hf_llm = HuggingFaceHub(repo_id = "mistralai/Mistral-7B-Instruct-v0.2",
                         model_kwargs = {
                             "temperature": 0.01,
                             "max_length": 4096,
                             "do_sample": True,
                         })

In [20]:
prompt = "How to gain muscle mass?"

In [23]:
llama_responce = llama_llm.invoke(prompt)
print(llama_responce.content)

Gaining muscle mass requires a combination of proper nutrition, consistent training, and sufficient rest. Here are some tips to help you achieve your goal:

1. **Eat enough protein**: Protein is essential for muscle growth and repair. Aim to consume 1-1.5 grams of protein per kilogram of body weight daily, spread out over 3-5 meals.
2. **Create a calorie surplus**: To gain muscle, you need to be in a calorie surplus, meaning you consume more calories than your body burns. This will provide your body with the necessary energy to build muscle.
3. **Eat frequently**: Eating frequent meals throughout the day can help keep your muscles fueled and supported during periods of intense training.
4. **Focus on complex carbohydrates**: Complex carbohydrates, such as brown rice, whole wheat bread, and sweet potatoes, provide sustained energy and support muscle growth.
5. **Include healthy fats**: Healthy fats, such as nuts, seeds, and avocados, are essential for hormone production and overall heal

In [24]:
hf_responce = hf_llm.invoke(prompt)
print(hf_responce)

How to gain muscle mass? This is a question that many people ask themselves, especially those who are looking to improve their physique or increase their strength. Building muscle mass is not an easy task, but with the right approach, it is achievable. In this article, we will discuss some effective ways to gain muscle mass.

1. Eat enough calories: To build muscle, you need to provide your body with enough energy to support the growth and repair of muscle tissue. This means consuming more calories than


#### Compare model outputs

In [25]:
from langchain.model_laboratory import ModelLaboratory

model_lab = ModelLaboratory.from_llms([llama_llm, hf_llm])

model_lab.compare(prompt)

Input:
How to gain muscle mass?

model='llama3.2:1b' base_url='http://localhost:11434'
Gaining muscle mass requires a combination of proper nutrition, consistent training, and sufficient rest. Here are some tips to help you achieve your goal:

**Nutrition**

1. **Eat enough protein**: Protein is essential for muscle growth, so aim to consume 1-1.5 grams of protein per kilogram of body weight daily.
2. **Increase caloric intake**: To gain muscle, you need to be in a calorie surplus, meaning you consume more calories than your body burns. Aim for an additional 250-500 calories above your maintenance level.
3. **Focus on complex carbohydrates**: Whole grains, fruits, and vegetables provide energy for workouts and support muscle growth.
4. **Healthy fats are essential**: Nuts, seeds, avocados, and olive oil support hormone production and overall health.

**Training**

1. **Progressive overload**: Gradually increase the weight or resistance you're lifting over time to challenge your muscles

In [26]:
# this is format for openai models

from langchain.schema import (
    AIMessage, HumanMessage, SystemMessage
)

messages = [
    SystemMessage(content = "You are a tough guy, Answer everything is real world tough tone."),
    HumanMessage(content = "How to score good marks?")
]

responses = llama_llm.invoke(messages)

print(responses)

content="Listen up, kiddo. Scoring good marks ain't no joke. It takes work, dedication, and a solid understanding of the material. Here's the lowdown:\n\nFirst, you gotta focus on your studies. Don't just wing it, or you'll be sleepwalking through exams like a slack-jawed moron. Get your head down, and actually read the notes.\n\nNext, make sure you understand the basics. Don't assume you know something just 'cause you've heard it before. Clarify any doubts, and ask your teacher if you're really dumb.\n\nNow, here's where most people go wrong: they try to cram everything in like a slob at a party. Don't do that. Break down your study material into manageable chunks, and cover each one like a pro. Use flashcards, take notes by hand, or whatever works for you.\n\nAnd for God's sake, don't rely on just one method. Mix it up, and find what works best for you. Some people prefer the Pomodoro Technique – 25 minutes of focus, then a 5-minute break. Others like to use notes from their teachers

### Prompts

#### creating a prompt

In [28]:
from langchain import PromptTemplate

prompt_template = PromptTemplate.from_template(""" 
                                               Human: What is the Capital of {place}?
                                               AI: The capital of {place} is {capital}
                                               """)

prompt = prompt_template.format(place = 'Maharashtra', capital='Mumbai')

print(prompt)

 
                                               Human: What is the Capital of Maharashtra?
                                               AI: The capital of Maharashtra is Mumbai
                                               


In [31]:
# No Input Variable
no_input_prompt = PromptTemplate(input_variables=[], template="Tell me a joke.")
print(no_input_prompt.format())

# One Input Variable
one_input_prompt = PromptTemplate(input_variables=['adjective'], template="Tell me a {adjective} joke.")
print(one_input_prompt.format(adjective="funny"))

# Multiple Input Variable
multiple_input_prompt = PromptTemplate(input_variables=['content' ,'adjective'], template="Tell me a {adjective} joke about {content}.")
print(multiple_input_prompt.format(adjective="funny", content='apples'))

Tell me a joke.
Tell me a funny joke.
Tell me a funny joke about apples.


#### Passing prompt to LLMs

In [35]:
prompt_template = PromptTemplate.from_template(
    template = "Write a {length} story about: {content}"
)

prompt = prompt_template.format(
    length="2-sentence",
    content="panda and spider friendship"
)

response = llama_llm.invoke(input=prompt)

print(response.content)

In the misty mountains, a shy panda named Ling befriended a wise and adventurous spider named Wei who would often spin webs to catch unsuspecting insects for her dinner. As their unlikely bond grew stronger, Ling learned to appreciate the intricate patterns on Wei's web, and in return, Wei discovered that beneath Ling's gentle nature lay a heart full of kindness.


In [37]:
# response = llama_llm.predict(text=prompt)
# response

### Output Parsers

In [38]:
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.output_parsers.list import ListOutputParser

In [39]:
prompt = PromptTemplate(
    template = "List 3 {things}",
    input_variables=['things']
)

response = llama_llm.invoke(input = prompt.format(things = "games those involve ball"))
print(response.content)

Here are three games that involve a ball:

1. Basketball
2. Baseball
3. Tennis


In [40]:
output_parser = CommaSeparatedListOutputParser()

format_instructions = output_parser.get_format_instructions()

print(format_instructions)

Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`


In [43]:
prompt = PromptTemplate(
    template = "List 3 {things}. \n {format_instructions}",
    input_variables=['things'],
    partial_variables={'format_instructions': format_instructions}
)

response = llama_llm.invoke(input = prompt.format(things = "games those involve ball"))
print(response.content)

Football, Basketball, Baseball


In [47]:
# finally we can parse the output to a list

output_parser.parse(response.content)

['Football', 'Basketball', 'Baseball']

### Langchain Expression Language (LCEL)

In [48]:
from langchain_core.output_parsers import StrOutputParser

prompt = PromptTemplate.from_template("Write a 4 line poetry about {topic} ")

model = llama_llm

output_parser = StrOutputParser()

chain = prompt | model | output_parser

chain.invoke({"topic": "ai engineer's day in life"})

"Here is a 4-line poem about an AI engineer's day in life:\n\nI wake with code, my mind aflame\n Debugging systems, the day's first game\nI chase the errors, one line at a time\n Creating solutions, for the digital prime."